In [107]:
# we want to build an AI text detector, that can detect if a text is written by an AI or a human

# 0. load packages
# 1. get the data
# 2. preprocess the data (embeddings)
# 3. build the model
# 4. train the model
# 5. evaluate the model
# 6. save the model

In [108]:
import numpy as np
import pandas as pd

In [109]:
# 1. get the data
filename = 'data.csv'
df = pd.read_csv(filename, header=0)
df

,text,is_generated
0,"Det gør firmaerne, der står bag AI'en - eksemp...",0
1,"Tror det bliver for kedeligt, hvis vi bare læs...",0
2,"Godt spørgsmål! Det er nemlig meget, meget svæ...",0
3,"Ja, det er begyndt at ske. Det er dog ikke så ...",1
4,Det vil jeg tro - uden at vide det helt præcis...,0
5,"Selvom det er svært at forudsige, hvornår AI v...",1
6,"Det afhænger nok meget af, hvor fremme i skoen...",0
7,"En tech-korrespondent er en journalist, der dæ...",1
8,"ChatGPT er en generativ AI-model, der lærer at...",1
9,"Hej, den har læst en masse tekst og så regner ...",0


In [116]:
# 2. preprocess the data (embeddings)
def get_embeddings(df):
        
    # build a vocabulary
    vocab = {}
    rvocab = {}
    c = 0
    for text in df['text']:
        for word in text.split():
            if word not in vocab:
                vocab[word] = c
                rvocab[c] = word
                c += 1

    # build the embeddings
    embeddings = np.random.rand(len(vocab), 10)


    # build the training data
    X = []

    for text in df['text']:
        x = []
        for word in text.split():
            x.append(embeddings[vocab[word]])
        X.append(np.array(x))

    # add padding
    maxlen = max([len(x) for x in X])
    for i in range(len(X)):
        X[i] = np.concatenate([X[i], np.zeros((maxlen - len(X[i]), 10))])

    X = np.array(X)

    y = df['is_generated'].values
    return X, y, vocab, rvocab, embeddings
X, y, vocab, rvocab, embeddings = get_embeddings(df)


In [113]:
# 3. build the model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, df):
        self.df = df

        X, y, vocab, rvocab, embeddings = get_embeddings(df)
        self.X = X
        self.y = torch.Tensor(y)
        self.len = len(y)
        
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len
    

# model should be transformer based
class TextModel(nn.Module):
    def __init__(self):
        super(TextModel, self).__init__()

        # embedding
        # self.embedding = nn.Embedding(10000, 10)
        
        
        # transformer encoder
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=10, nhead=2, dim_feedforward=1048, dropout=0.1, activation='relu'
            ),
            num_layers=6
        )

        # classifier
        self.classifier = nn.Sequential(
            nn.Linear(163, 100),
            nn.LeakyReLU(),
            nn.Linear(100, 1), # should maybe be 2
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x


# 3. train the model
model = TextModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

dataset = TextDataset(df)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


for epoch in range(100):
    loss_accum = 0
    for i, (X, y) in enumerate(dataloader):
        X = X.squeeze(0)
        # convert to float 32
        X = X.float()
        optimizer.zero_grad()
        y_pred = model(X)
        loss = criterion(y_pred, y)
        loss.backward()
        optimizer.step()
        loss_accum += loss.item()
    if epoch % 10 == 0:
        print(f'Epoch {epoch} loss: {loss_accum/len(dataloader)}')



Epoch 0 loss: 0.7021608889102936
Epoch 10 loss: 0.6992170155048371
Epoch 20 loss: 0.6904897034168244
Epoch 30 loss: 0.6713377237319946
Epoch 40 loss: 0.6405147790908814
Epoch 50 loss: 0.5772414594888687
Epoch 60 loss: 0.5189620643854141
Epoch 70 loss: 0.45294628739356996
Epoch 80 loss: 0.3704911023378372
Epoch 90 loss: 0.3028962671756744


In [120]:
# 5. evaluate the model

text = 'Jeg er fucking ikke en AI'

# convert text to embedding
x = []
for word in text.split():
    x.append(embeddings[vocab[word]])
x = np.array(x)

# add padding
maxlen = 163
x = np.concatenate([x, np.zeros((maxlen - len(x), 10))])
x = np.array([x])

# convert to float 32
x = torch.Tensor(x).float()
# predict
y_pred = model(x.squeeze(0))

print(y_pred)


KeyError: 'fucking'